Km2

In [15]:
# -*- coding: utf-8 -*-
from pathlib import Path
import re
import unicodedata
import pandas as pd

# === CONFIG ===
INPUT  = Path(r"D:\ARTIGOS\2025\LARANJA_MAX\04_SAIDA_MAXENT\processamento\7_excel")   # arquivos completos
OUTPUT = Path(r"D:\ARTIGOS\2025\LARANJA_MAX\07_POS_ANALISE\15_PAISES")               # onde salvar filtrados
SHEET_NAME = "resumo"
OUTPUT.mkdir(parents=True, exist_ok=True)

# ORDEM FINAL (inglês, exatamente como você pediu)
COUNTRIES_EN = [
    "Brazil",
    "China",
    "Mexico",
    "India",
    "United States of America",
    "Spain",
    "Egypt",
    "Italy",
    "Iraq",
]

# aliases comuns -> nome canônico (inglês)
ALIASES = {
    # EUA
    "USA": "United States of America",
    "U.S.": "United States of America",
    "United States": "United States of America",
    "United States of America": "United States of America",
    # Egito
    "Egypt": "Egypt",
    "Arab Republic of Egypt": "Egypt",
    "Egypt, Arab Rep.": "Egypt",
    # Espanha
    "Spain": "Spain",
    "Kingdom of Spain": "Spain",
    # Brasil, China, México, Índia, Itália, Iraque
    "Brazil": "Brazil",
    "Brasil": "Brazil",
    "China": "China",
    "Mexico": "Mexico",
    "Estados Unidos Mexicanos": "Mexico",
    "India": "India",
    "Italy": "Italy",
    "Iraq": "Iraq",
}

def strip_accents(s: str) -> str:
    return "".join(c for c in unicodedata.normalize("NFD", s) if unicodedata.category(c) != "Mn")

def normalize_text(x: object) -> str:
    s = str(x).strip()
    s = re.sub(r"\s+", " ", s)
    return s

def canonize_country(name: str) -> str:
    n = normalize_text(name)
    if n in ALIASES:
        return ALIASES[n]
    # tenta remover parênteses
    n2 = re.sub(r"\s*\(.*?\)\s*", "", n).strip()
    if n2 in ALIASES:
        return ALIASES[n2]
    # compara sem acento/maiúsculas
    n_key = strip_accents(n).lower()
    for k, v in ALIASES.items():
        if strip_accents(k).lower() == n_key:
            return v
    return n  # se não mapear, retorna como veio

def load_one(xlsx: Path) -> pd.DataFrame:
    df = pd.read_excel(xlsx, sheet_name=SHEET_NAME)
    # remove colunas "Unnamed"
    df = df.loc[:, ~df.columns.astype(str).str.contains(r"^Unnamed")]
    if "name" not in df.columns:
        raise ValueError(f"{xlsx.name}: coluna 'name' ausente na aba '{SHEET_NAME}'.")
    # normaliza nomes
    df["name"] = df["name"].map(canonize_country)
    return df

for xlsx in sorted(INPUT.glob("*.xlsx")):
    try:
        df = load_one(xlsx)
    except Exception as e:
        print(f"⚠️  {xlsx.name}: erro ao abrir/ler. Pulando. ({e})")
        continue

    # filtra apenas países de interesse; mantém múltiplas linhas (ex.: por grid)
    df_filt = df[df["name"].isin(COUNTRIES_EN)].copy()
    # ordena pela ordem desejada (mais gridcode se existir)
    df_filt["__ordem__"] = pd.Categorical(df_filt["name"], categories=COUNTRIES_EN, ordered=True)
    sort_cols = ["__ordem__"] + (["gridcode"] if "gridcode" in df_filt.columns else [])
    df_filt = df_filt.sort_values(sort_cols).drop(columns="__ordem__")

    # relatório
    encontrados = df_filt["name"].drop_duplicates().tolist()
    faltantes = [p for p in COUNTRIES_EN if p not in encontrados]

    # salva com o MESMO nome do arquivo original
    destino = OUTPUT / xlsx.name
    try:
        with pd.ExcelWriter(destino, engine="openpyxl", mode="w") as writer:
            df_filt.to_excel(writer, sheet_name=SHEET_NAME, index=False)
        print(
            f"✅  {xlsx.name}: {len(encontrados)}/{len(COUNTRIES_EN)} países; "
            f"{len(df_filt)} linhas salvas em '{destino.name}'."
            + (f" Faltantes: {', '.join(faltantes)}." if faltantes else "")
        )
    except Exception as e:
        print(f"❌  {xlsx.name}: erro ao salvar. ({e})")


✅  2021_2040_ssp126.xlsx: 9/9 países; 36 linhas salvas em '2021_2040_ssp126.xlsx'.
✅  2021_2040_ssp245.xlsx: 9/9 países; 9 linhas salvas em '2021_2040_ssp245.xlsx'.
✅  2021_2040_ssp370.xlsx: 9/9 países; 9 linhas salvas em '2021_2040_ssp370.xlsx'.
✅  2021_2040_ssp585.xlsx: 9/9 países; 9 linhas salvas em '2021_2040_ssp585.xlsx'.
✅  2041_2060_ssp126.xlsx: 9/9 países; 9 linhas salvas em '2041_2060_ssp126.xlsx'.
✅  2041_2060_ssp245.xlsx: 9/9 países; 9 linhas salvas em '2041_2060_ssp245.xlsx'.
✅  2041_2060_ssp370.xlsx: 9/9 países; 9 linhas salvas em '2041_2060_ssp370.xlsx'.
✅  2041_2060_ssp585.xlsx: 9/9 países; 9 linhas salvas em '2041_2060_ssp585.xlsx'.
✅  2061_2080_ssp126.xlsx: 9/9 países; 9 linhas salvas em '2061_2080_ssp126.xlsx'.
✅  2061_2080_ssp245.xlsx: 9/9 países; 9 linhas salvas em '2061_2080_ssp245.xlsx'.
✅  2061_2080_ssp370.xlsx: 9/9 países; 9 linhas salvas em '2061_2080_ssp370.xlsx'.
✅  2061_2080_ssp585.xlsx: 9/9 países; 9 linhas salvas em '2061_2080_ssp585.xlsx'.
✅  2081_2100_ss

In [25]:
# -*- coding: utf-8 -*-
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
import re, unicodedata, sys

# ========================= CONFIG =========================
INPUT  = Path(r"D:\ARTIGOS\2025\LARANJA_MAX\04_SAIDA_MAXENT\processamento\7_excel")
FILES = {
    "Actual":    INPUT / "actual.xlsx",
    "SSP1-2.6":  INPUT / "2081_2100_ssp126.xlsx",
    "SSP2-4.5":  INPUT / "2081_2100_ssp245.xlsx",
    "SSP3-7.0":  INPUT / "2081_2100_ssp370.xlsx",
    "SSP5-8.5":  INPUT / "2081_2100_ssp585.xlsx",
}
SCENARIOS = ["Actual","SSP1-2.6","SSP2-4.5","SSP3-7.0","SSP5-8.5"]

# >>> ORDEM FINAL (inglês) atualizada
COUNTRIES = [
    "Brazil",
    "China",
    "Mexico",
    "India",
    "United States of America",
    "Egypt",
    "Spain",
    "Iraq",
    "Italy",
    "Indonesia",
]

# Aliases -> nome canônico (inglês)
ALIASES = {
    # básicos
    "Brazil":"Brazil","Brasil":"Brazil",
    "China":"China",
    "Mexico":"Mexico","Estados Unidos Mexicanos":"Mexico",
    "India":"India",
    "Italy":"Italy","Italia":"Italy",
    "Iraq":"Iraq","Iraqi":"Iraq",
    "Indonesia":"Indonesia","Rep. Indonesia":"Indonesia","Republic of Indonesia":"Indonesia",
    # EUA
    "United States of America":"United States of America",
    "United States":"United States of America",
    "USA":"United States of America","U.S.":"United States of America",
    # Espanha
    "Spain":"Spain","Kingdom of Spain":"Spain","España":"Spain",
    # Egito
    "Egypt":"Egypt","Arab Republic of Egypt":"Egypt","Egypt, Arab Rep.":"Egypt",
}

OUTDIR = Path(r"D:\ARTIGOS\2025\LARANJA_MAX\07_POS_ANALISE\APTIDAO_CLASSES")
OUTDIR.mkdir(parents=True, exist_ok=True)
OUT_XLSX = OUTDIR / "APTIDAO_CLASSES_10_en_km2_pct.xlsx"
OUT_PNG  = OUTDIR / "grouped_actual_SSPs_2081-2100_EN_compact_noLegend_noNames.png"

COLS = {"Unsuitable":"#BAADAD","Low":"#fdae61","Moderate":"#f46d43","Highly":"#d73027"}

# ========================= HELPERS =========================
def strip_accents(s: str) -> str:
    return "".join(c for c in unicodedata.normalize("NFD", s) if unicodedata.category(c) != "Mn")

def norm_text(x: object) -> str:
    s = str(x).strip()
    s = re.sub(r"\s+", " ", s)
    return s

def canonize_country(name: str) -> str:
    n = norm_text(name)
    if n in ALIASES: return ALIASES[n]
    n2 = re.sub(r"\s*\(.*?\)\s*", "", n).strip()
    if n2 in ALIASES: return ALIASES[n2]
    n_key = strip_accents(n).lower()
    for k, v in ALIASES.items():
        if strip_accents(k).lower() == n_key: return v
    return n

def read_one(path: Path, sheet="resumo") -> pd.DataFrame:
    df = pd.read_excel(path, sheet_name=sheet)
    df = df.loc[:, ~df.columns.astype(str).str.contains(r"^Unnamed")]
    if "name" not in df.columns:
        raise ValueError(f"{path.name}: coluna 'name' ausente na aba '{sheet}'.")
    df["name"] = df["name"].map(canonize_country)
    # MapA (correto): 1=Unsuitable, 2=Low, 3=Moderate, 4=Highly
    df = df.rename(columns={1:"Unsuitable", 2:"Low", 3:"Moderate", 4:"Highly"})
    for c in ["Unsuitable","Low","Moderate","Highly"]:
        if c not in df.columns: df[c] = 0
        df[c] = pd.to_numeric(df[c], errors="coerce").fillna(0)
    # agrega por país, filtra e reordena na lista COUNTRIES
    df = (df.groupby("name", as_index=False)[["Unsuitable","Low","Moderate","Highly"]].sum()
            .set_index("name").reindex(COUNTRIES).fillna(0).reset_index())
    # percentuais
    df["TOTAL"] = df[["Unsuitable","Low","Moderate","Highly"]].sum(axis=1)
    for c in ["Unsuitable","Low","Moderate","Highly"]:
        df[c+"_pct"] = np.where(df["TOTAL"]>0, df[c]/df["TOTAL"]*100, 0)
    return df

def to_long(df_agg: pd.DataFrame, scenario: str) -> pd.DataFrame:
    km2 = df_agg.melt(id_vars=["name"], value_vars=["Unsuitable","Low","Moderate","Highly"],
                      var_name="class", value_name="km2")
    pct = df_agg.melt(id_vars=["name"], value_vars=["Unsuitable_pct","Low_pct","Moderate_pct","Highly_pct"],
                      var_name="class_pct", value_name="pct")
    pct["class"] = pct["class_pct"].str.replace("_pct","",regex=False)
    pct = pct.drop(columns=["class_pct"])
    long = km2.merge(pct, on=["name","class"], how="inner")
    long.insert(1, "scenario", scenario)
    long["name"]  = pd.Categorical(long["name"],  categories=COUNTRIES, ordered=True)
    long["class"] = pd.Categorical(long["class"], categories=["Unsuitable","Low","Moderate","Highly"], ordered=True)
    return long.sort_values(["name","class"]).reset_index(drop=True)

# ========================= RUN (LER, CONSOLIDAR) =========================
print("==> Lendo cenários e consolidando…")
all_long = []
for scen, path in FILES.items():
    if not path.exists():
        print(f"✖ Arquivo não encontrado: {path}")
        continue
    print(f"✔ {scen}: {path.name}")
    df = read_one(path)
    all_long.append(to_long(df, scen))

if not all_long:
    sys.exit("Nenhum cenário válido lido. Verifique caminhos e nomes dos .xlsx em INPUT.")
all_long = pd.concat(all_long, ignore_index=True).round(2)
print("✔ Consolidação concluída.")

# ========================= SALVAR EXCEL =========================
with pd.ExcelWriter(OUT_XLSX, engine="openpyxl") as xl:
    all_long.to_excel(xl, sheet_name="long_km2_pct", index=False)
    for scen in SCENARIOS:
        sub = all_long[all_long["scenario"]==scen]
        if sub.empty: 
            continue
        wide_km2 = sub.pivot(index="name", columns="class", values="km2").reindex(COUNTRIES)
        wide_pct = sub.pivot(index="name", columns="class", values="pct").reindex(COUNTRIES).round(2)
        wide_km2.to_excel(xl, sheet_name=f"{scen}_km2")
        wide_pct.to_excel(xl, sheet_name=f"{scen}_pct")
print(f"✅ Excel salvo: {OUT_XLSX}")

# ========================= GRÁFICO (SEM NOMES) =========================
def plot_compact_no_country_labels(df_long: pd.DataFrame, scenarios, countries, colors, save_png: Path):
    rows = []
    for nm in countries:
        for sc in scenarios:
            sub = df_long[(df_long["name"]==nm) & (df_long["scenario"]==sc)]
            vals = {c: float(sub.loc[sub["class"]==c, "pct"].values[0]) if (sub["class"]==c).any() else 0.0
                    for c in ["Unsuitable","Low","Moderate","Highly"]}
            rows.append([nm, sc, vals["Unsuitable"], vals["Low"], vals["Moderate"], vals["Highly"]])
    dd = pd.DataFrame(rows, columns=["name","scenario","Unsuitable_pct","Low_pct","Moderate_pct","Highly_pct"])

    plt.rcParams.update({"figure.dpi": 120, "font.size": 10.4, "axes.labelsize": 10.4})
    figsize=(6.3,7.9)  # 10 países
    inner_gap=0.43; country_gap=0.78; left_limit=-140; right_limit=100
    y_pos, centers = [], []; y=0.0; nsc=len(scenarios)

    for _ in countries:
        for _ in range(nsc): y_pos.append(y); y += inner_gap
        centers.append(y - (nsc*inner_gap)/2)
        y += country_gap
    y_pos = np.array(y_pos)

    fig, ax = plt.subplots(figsize=figsize)
    h=0.47
    imp = -dd["Unsuitable_pct"].to_numpy()
    low =  dd["Low_pct"].to_numpy()
    mod =  dd["Moderate_pct"].to_numpy()
    hig =  dd["Highly_pct"].to_numpy()
    ax.barh(y_pos, imp, color=colors["Unsuitable"], height=h)
    ax.barh(y_pos, low, color=colors["Low"], height=h)
    ax.barh(y_pos, mod, left=low, color=colors["Moderate"], height=h)
    ax.barh(y_pos, hig, left=low+mod, color=colors["Highly"], height=h)

    ax.axvline(0, color="#4f4f4f", linewidth=0.8)
    ax.set_xlim(left_limit, right_limit)
    ax.grid(axis="x", linewidth=0.3, alpha=0.45)
    ax.set_xlabel("Share of country area (%)")

    ax.set_yticks([])  # SEM nomes
    for yc in centers:
        ax.hlines(y=yc + (nsc*inner_gap/2), xmin=left_limit, xmax=right_limit,
                  colors="#e6e6e6", linewidth=0.75)

    ax.invert_yaxis()
    fig.subplots_adjust(left=0.04, right=0.98, top=0.98, bottom=0.08)
    fig.savefig(save_png, dpi=1000, bbox_inches="tight")
    plt.close(fig)
    print(f"🖼️  PNG salvo: {save_png}")

plot_compact_no_country_labels(
    df_long=all_long,
    scenarios=SCENARIOS,
    countries=COUNTRIES,
    colors=COLS,
    save_png=OUT_PNG
)


==> Lendo cenários e consolidando…
✔ Actual: actual.xlsx
✔ SSP1-2.6: 2081_2100_ssp126.xlsx
✔ SSP2-4.5: 2081_2100_ssp245.xlsx
✔ SSP3-7.0: 2081_2100_ssp370.xlsx
✔ SSP5-8.5: 2081_2100_ssp585.xlsx
✔ Consolidação concluída.
✅ Excel salvo: D:\ARTIGOS\2025\LARANJA_MAX\07_POS_ANALISE\APTIDAO_CLASSES\APTIDAO_CLASSES_10_en_km2_pct.xlsx
🖼️  PNG salvo: D:\ARTIGOS\2025\LARANJA_MAX\07_POS_ANALISE\APTIDAO_CLASSES\grouped_actual_SSPs_2081-2100_EN_compact_noLegend_noNames.png
